In [1]:
import numpy as np

In [2]:
# Used for the Cython magic inside ipynb
%load_ext cython

## Vanilla Python version

In [3]:
def clip_py(a, min_value, max_value):
    return min(max(a, min_value), max_value)


def compute_py(array_1, array_2, a, b, c):
    """
    This function must implement the formula
    np.clip(array_1, 2, 10) * a + array_2 * b + c

    array_1 and array_2 are 2D.
    """
    x_max = array_1.shape[0]
    y_max = array_1.shape[1]

    assert array_1.shape == array_2.shape

    result = np.zeros((x_max, y_max), dtype=array_1.dtype)

    for x in range(x_max):
        for y in range(y_max):
            tmp = clip_py(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result[x, y] = tmp + c

    return result

## Advanced Cython

In [4]:
%%cython

import numpy as np
# We now need to fix a datatype for our arrays. I've used the variable
# DTYPE for this, which is assigned to the usual NumPy runtime
# type info object.
DTYPE = np.intc

# cdef means here that this function is a plain C function (so faster).
# To get all the benefits, we type the arguments and the return value.
cdef int clip_cy(int a, int min_value, int max_value):
    return min(max(a, min_value), max_value)

def compute_cy(int[:, :] array_1, int[:, :] array_2, int a, int b, int c):

    cdef Py_ssize_t x_max = array_1.shape[0]
    cdef Py_ssize_t y_max = array_1.shape[1]

    # array_1.shape is now a C array, no it's not possible
    # to compare it simply by using == without a for-loop.
    # To be able to compare it to array_2.shape easily,
    # we convert them both to Python tuples.
    assert tuple(array_1.shape) == tuple(array_2.shape)

    result = np.zeros((x_max, y_max), dtype=DTYPE)
    cdef int[:, :] result_view = result

    cdef int tmp
    cdef Py_ssize_t x, y

    for x in range(x_max):
        for y in range(y_max):

            tmp = clip_cy(array_1[x, y], 2, 10)
            tmp = tmp * a + array_2[x, y] * b
            result_view[x, y] = tmp + c

    return result

## Comparison Python-Cython-NumPy

In [5]:
array_1 = np.random.uniform(0, 500, size=(1500, 1000)).astype(np.intc)
array_2 = np.random.uniform(0, 500, size=(1500, 1000)).astype(np.intc)
a, b, c = 4, 3, 9

import time

def compute_np(array_1, array_2, a, b, c):
    return np.clip(array_1, 2, 10) * a + array_2 * b + c

t1 = time.time()
compute_py(array_1, array_2, a, b, c)
t2 = time.time()

t3 = time.time()
compute_cy(array_1, array_2, a, b, c)
t4 = time.time()

t5 = time.time()
compute_np(array_1, array_2, a, b, c)
t6 = time.time()

print("Python: {}, Cython: {}, Numpy: {}".format(t2-t1,t4-t3, t6-t5))

Python: 15.361252784729004, Cython: 0.004180908203125, Numpy: 0.00778508186340332
